In [40]:
import numpy as np
import plotly.graph_objs as go
from time import sleep
import serial
from scipy.fft import rfft, rfftfreq
from plotly.subplots import make_subplots
import asyncio
import nest_asyncio

nest_asyncio.apply()

stream = serial.Serial()
stream.baudrate = 9600
stream.port = "/dev/ttyACM1"
stream.timeout = 0.01
stream.open()

rate = 50
dur = 1

d = list(range(0, rate * dur))

def get_sample():
    #stream.reset_input_buffer()
    global d
    sleep(1 / rate)
    try:
        s = stream.readline().decode()
        s = float(s)
        s = s / 1024.
        #if (s > 0.15 and s < 0.95):
        d.append(s)
        if (len(d) > (rate * dur)):
            d.pop(0)
    except Exception as e:
        pass#print(e)

fig = go.FigureWidget(make_subplots(rows=3, cols=1))

fig.append_trace(go.Scatter(x=None, y=None,
                    mode='lines',
                    name='lines',
                    line_shape='linear'),
             row=1, col=1)

fig.append_trace(go.Scatter(x=None, y=None,
                    mode='lines',
                    name='lines',
                    line_shape='linear'),
             row=2, col=1)

fig.append_trace(go.Bar(x=["Alpha", "Beta", "Theta"], y=[0, 0, 0]),
             row=3, col=1)

fig.update_layout(    
    {    
        'xaxis':{'range': [4, 20]},
        'yaxis':{'range': [-0.1, 0.5]},
        #'yaxis2':{'range': [0.81, 0.85]},
        'yaxis3':{'range': [0.0, 0.5]},
    })
fig

FigureWidget({
    'data': [{'line': {'shape': 'linear'},
              'mode': 'lines',
              'name':…

In [ ]:
import threading

_read_lock = threading.Lock()
_write_lock = threading.Lock()

n_a_s = [0, 0, 0]
n_b_s = [0, 0, 0]
n_t_s = [0, 0, 0]

def fft_thread():
    global n_a_s, n_b_s, n_t_s
    #with _read_lock:
    #    internal_d = d[:]

    yf = rfft(d)
    np_a = np.abs(yf)
    #np_a /= np.max(np_a[4*2:20*2])
    xf = rfftfreq(rate * dur, 1 / rate)
    alpha = np_a[8*2:12*2] / 8.
    a_s = sum(alpha)
    beta = np_a[14*2:20*2] / 12.
    b_s = sum(beta)
    theta = np_a[4*2:8*2] / 8.
    t_s = sum(theta)

    n_a_s.append(a_s)
    n_a_s = n_a_s[-3:]
    a_s = sum(n_a_s)

    n_b_s.append(b_s)
    n_b_s = n_b_s[-3:]
    b_s = sum(n_b_s)

    n_t_s.append(t_s)
    n_t_s = n_t_s[-3:]
    t_s = sum(n_t_s)

    s = a_s + b_s + t_s

    a_s = a_s / s
    b_s = b_s / s
    t_s = t_s / s

    #with _write_lock:
    fig.data[0].x = xf
    fig.data[0].y = np_a
    fig.data[2].y = [a_s, b_s, t_s]
        
t = threading.Thread(target=fft_thread)
        
n = 0
while True:
    #with _read_lock:
    get_sample()
    #internal_d = d[:]
    with _write_lock:
        fig.data[1].y = d
    if n == 10 and not t.is_alive():
        #t = threading.Thread(target=fft_thread)
        #t.start()
        fft_thread()
        n = 0
    n += 1